<a href="https://colab.research.google.com/github/codeRSH/AAG-DS/blob/main/AAG_Exploratory_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Analysis using Python

## Initial Setup

### Mount Drive to Read files

In [139]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import All Relevant Libraries

In [151]:
import glob
import pandas as pd
import numpy as np
import warnings

### Ignore Warnings (for clean output)

In [141]:
warnings.filterwarnings('ignore')

### Define File Paths

In [143]:
init_input_path = "/content/drive/MyDrive/AAG DS Folder/Capstone/input/"
init_output_path = "/content/drive/MyDrive/AAG DS Folder/Capstone/output/"

linfol_path = init_input_path + "LINFOL/*CSV"
kna1_path = init_input_path + "KNA1/*csv"
hueres_path = init_input_path + "HUERES/*csv"
rsrvs_path =  init_input_path + "RSRVS/*csv"

## Data Loading and Wrangling

### LINFOL - Hotel Revenue Data

In [150]:
# Linfol Data
linfol_data = pd.DataFrame()

# print(glob.glob(kna1_path))

# Read the LINFOL data from all the available csv files
for file in glob.glob(linfol_path):
    data = pd.read_csv(file, parse_dates=True, encoding="unicode_escape", sep='\t', 
                       header=None, skiprows=[0],
                       error_bad_lines=False, warn_bad_lines=False)
    linfol_data = linfol_data.append(data)

In [145]:
# Remove onwands columns. Also, keep a copy so as to not have to re-read the data. 
linfol_copy = linfol_data.drop(columns=[2, 4])
linfol_data = linfol_copy

In [146]:
# Provide the Column Names
linfol_data.columns = ["Hotel_ID", "Reservation_ID", "Concept", "Revenue_Date", "Revenue", "Tax"]
linfol_data.head()

,Hotel_ID,Reservation_ID,Concept,Revenue_Date,Revenue,Tax
0,AGR1,3750093.0,BREAKFAST,24.07.2018,400,72
1,AGR1,3750093.0,SERVCHRG,24.07.2018,120,21.6
2,AGR1,3750093.0,ALLOWFNB,26.07.2018,519.200000+AC0-,0
3,AGR1,3750102.0,BREAKFAST,24.07.2018,400,72
4,AGR1,3750102.0,SERVCHRG,24.07.2018,120,21.6


In [149]:
# Determine null values in each columns
linfol_data.isna( ).mean().round(4) * 100

Hotel_ID          0.0
Reservation_ID    0.0
Concept           0.0
Revenue_Date      0.0
Revenue           0.0
Tax               0.0
dtype: float64

In [148]:
# Convert Null Reservation ID to 0 and convert it to Integer
linfol_data = linfol_data[linfol_data['Reservation_ID'].notna()]
linfol_data["Reservation_ID"] = linfol_data["Reservation_ID"].astype(int)
linfol_data.head()

,Hotel_ID,Reservation_ID,Concept,Revenue_Date,Revenue,Tax
0,AGR1,3750093,BREAKFAST,24.07.2018,400,72
1,AGR1,3750093,SERVCHRG,24.07.2018,120,21.6
2,AGR1,3750093,ALLOWFNB,26.07.2018,519.200000+AC0-,0
3,AGR1,3750102,BREAKFAST,24.07.2018,400,72
4,AGR1,3750102,SERVCHRG,24.07.2018,120,21.6


In [10]:
# Remove +AC0- which is a wrong text in Revenue and Tax fields
linfol_data['Revenue'] = linfol_data['Revenue'].apply(lambda x : np.float(
                            x.replace("+AC0-", "").replace(",", "") 
                            if "+AC0-" in str(x) else np.float(x)))
linfol_data['Tax']     = linfol_data['Tax'].apply(lambda x : np.float(
                            x.replace("+AC0-", "").replace(",", "") 
                            if "+AC0-" in str(x) else np.float(x)))

# Create Total Column and remove the Revenue and Tax columns
linfol_data['Total'] = linfol_data['Revenue'] + linfol_data['Tax']
linfol_data.drop(columns = ["Revenue", "Tax"], inplace = True)
linfol_data.head()

,Hotel_ID,Reservation_ID,Concept,Revenue_Date,Total
0,AGR1,3750093,BREAKFAST,24.07.2018,472.0
1,AGR1,3750093,SERVCHRG,24.07.2018,141.6
2,AGR1,3750093,ALLOWFNB,26.07.2018,519.2
3,AGR1,3750102,BREAKFAST,24.07.2018,472.0
4,AGR1,3750102,SERVCHRG,24.07.2018,141.6


In [137]:
# Again check the null values
linfol_data.isna( ).mean().round(4) * 100

Hotel_ID          0.0
Reservation_ID    0.0
Concept           0.0
Revenue_Date      0.0
Total             0.0
dtype: float64

### KNA1 - Customer Data

In [20]:
# KNA1 Data
kna1_data = pd.DataFrame()

# print(glob.glob(kna1_path))
# Read the KNA1 data from all the available csv files
for file in glob.glob(kna1_path):
    data = pd.read_csv(file, parse_dates=True, encoding="latin1",
                       error_bad_lines=False, usecols = [0, 1, 2, 3, 4, 6, 12, 29] )
    kna1_data = kna1_data.append(data)

kna1_data.columns = ["Cust_ID", "Country_ID", "Name", "Name2", "City", "Region", "Gender", "Company"]

In [42]:
kna1_data.isna( ).mean().round(4) * 100

Cust_ID       0.00
Country_ID    0.00
Name          0.04
City          1.59
Region        4.17
Gender        0.00
Company       0.00
dtype: float64

In [21]:
# Convert Null to blank in Name2 and combine Name1 and Name2 fields to get full name. Drop Name2
kna1_data["Name2"].fillna( value = " ", inplace = True)
kna1_data["Name"] = kna1_data["Name"] + kna1_data["Name2"]
kna1_data.drop(columns=["Name2"], axis = 1, inplace=True)

,Cust_ID,Country_ID,Name,City,Region,Gender,Company
0,1,IN,N.k Bhandari,Ahmedabad,13,NaN,NaN
1,2,IN,Sandip Laxman,Maharashtra,13,NaN,NaN
2,3,IN,Sushmita Choudhary,Jaipur,20,NaN,NaN
3,7,IN,RAJKUMAR SHAH,AHMEDABAD,6,Mr.,ROOTS CORPORATION LIMITED
4,8,IN,Vasanti Gokal,Mumbai,13,Ms.,TECHNOSALE


In [25]:
# 99% Company values are null. Make them blank
kna1_data["Company"].fillna(value = " " , inplace = True )
kna1_data.head()

,Cust_ID,Country_ID,Name,City,Region,Gender,Company
0,1,IN,N.k Bhandari,Ahmedabad,13,NaN,
1,2,IN,Sandip Laxman,Maharashtra,13,NaN,
2,3,IN,Sushmita Choudhary,Jaipur,20,NaN,
3,7,IN,RAJKUMAR SHAH,AHMEDABAD,6,Mr.,ROOTS CORPORATION LIMITED
4,8,IN,Vasanti Gokal,Mumbai,13,Ms.,TECHNOSALE


In [43]:
# Determine the Percentage of Males
kna1_data[kna1_data["Gender"] == "Mr."]["Gender"].count( )/(len(kna1_data["Gender"]))  * 100

82.03851140323421

In [44]:
# Determine the  Percentage of Females
kna1_data[kna1_data["Gender"] == "Ms."]["Gender"].count( )/(len(kna1_data["Gender"]))  * 100

17.460668543425307

In [45]:
# For now consider all null values as Males
kna1_data["Gender"].fillna(value = "Mr.", inplace = True )

In [49]:
# Count the number of unmarried females (Ms.)
kna1_data[(kna1_data["Gender"] != "Mr.") & (kna1_data["Gender"] != "Mrs.") ]["Gender"].count( )

181903

### HUERES - Guest/Reservation ID Combination Data 

In [52]:
# HUERES Data
hueres_data = pd.DataFrame()

# print(glob.glob(hueres_path))
# Read the HUERES data from all the available csv files
for file in glob.glob(hueres_path):
    data = pd.read_csv(file, parse_dates=True, encoding="latin1",
                       error_bad_lines=False, usecols = [0, 1, 2, 3] )
    hueres_data = hueres_data.append(data)

hueres_data.columns = ["Hotel_ID", "Reservation_ID", "Pers_Pos", "Guest_ID"]
hueres_data.head()

['/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Feb to 28th Feb 2019.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Aug to 31st Aug 2017.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Aug to 31st Aug 2018.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Feb to 28th Feb 2018.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Jan to 31st Jan 2019.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Apr to 30th Apr 2018.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Apr to 30th Apr 2019.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Dec to 3st Dec 2018.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone/input/HUERES/Copy of Hueres 1st Dec to 31st Dec 2017.csv', '/content/drive/MyDrive/AAG DS Folder/Capstone

,Hotel_ID,Reservation_ID,Pers_pos,Guest_ID
0,LKO2,3338617,1,NaN
1,LKO2,3338617,2,NaN
2,LKO2,3338620,1,NaN
3,LKO2,3338620,2,NaN
4,LKO2,3338621,1,NaN


In [53]:
# Find the percentage of Nulls in each column
hueres_data.isna( ).mean().round(4) * 100

Hotel_ID           0.00
Reservation_ID     0.00
Pers_pos           0.00
Guest_ID          14.03
dtype: float64

In [54]:
# Make Null Guest ID blank and convert it into integer
hueres_data["Guest_ID"].fillna( value = 0, inplace = True)
hueres_data["Guest_ID"] = hueres_data["Guest_ID"].astype(int)
hueres_data.head()

,Hotel_ID,Reservation_ID,Pers_pos,Guest_ID
0,LKO2,3338617,1,0
1,LKO2,3338617,2,0
2,LKO2,3338620,1,0
3,LKO2,3338620,2,0
4,LKO2,3338621,1,0


### RSRVS - Reservation Data

In [16]:
# RSRVS Data
rsrvs_data = pd.DataFrame()

# print(glob.glob(rsrvs_path))
# Read the RSRVS data from all the available csv files
for file in glob.glob(rsrvs_path):
    data = pd.read_csv(file, parse_dates=True, encoding="latin1",
                       error_bad_lines=False, usecols = [0, 1, 2, 3, 4, 7, 12, 13, 16, 18, 24, 25, 26] )
    rsrvs_data = rsrvs_data.append(data)

rsrvs_data.columns = ["Hotel_ID", "Reservation_ID", "Main_Cust_ID",  "Checkin_Date", "Checkout_Date", "PAX", "Room_Type", "Upg_or_Real", "Reservation_Status", "Board_ID", "Cancel_Date", "Cancel_Rsn_ID", "Reg_Date"]
rsrvs_data.head()

,Hotel_ID,Reservation_ID,Main_Cust_ID,Checkin_Date,Checkout_Date,PAX,Room_Type,Upg_or_Real,Reservation_Status,Board_ID,Cancel_Date,Cancel_Rsn_ID,Reg_Date
0,AGR1,3681294,90003186.0,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
1,AGR1,3681296,90003186.0,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
2,AGR1,3681298,90003186.0,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
3,AGR1,3681299,90003186.0,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
4,AGR1,3681300,90003186.0,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018


In [17]:
# Convert Null Main Customer ID to blank and convert it to integer. 
rsrvs_data["Main_Cust_ID"].fillna(value = 0, inplace = True)
rsrvs_data["Main_Cust_ID"] = rsrvs_data["Main_Cust_ID"].astype(int)
rsrvs_data.head()

,Hotel_ID,Reservation_ID,Main_Cust_ID,Checkin_Date,Checkout_Date,PAX,Room_Type,Upg_or_Real,Reservation_Status,Board_ID,Cancel_Date,Cancel_Rsn_ID,Reg_Date
0,AGR1,3681294,90003186,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
1,AGR1,3681296,90003186,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
2,AGR1,3681298,90003186,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
3,AGR1,3681299,90003186,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018
4,AGR1,3681300,90003186,16.02.2019,17.02.2019,2,DBL,NaN,6,ONLYROOM,23.01.2019,1,05.06.2018


In [64]:
# Find the percentage of Nulls in each column
rsrvs_data.isna( ).mean().round(4) * 100

Hotel_ID              0.00
Reservation_ID        0.00
Main_Cust_ID          0.00
Checkin_Date          0.00
Checkout_Date         0.00
PAX                   0.00
Room_Type             0.50
Upg_or_Real           0.00
Reservation_Status    0.00
Board_ID              0.49
Cancel_Date           0.00
Cancel_Rsn_ID         0.00
Reg_Date              0.00
dtype: float64

In [63]:
# This doesn't give error but not working either!
# rsrvs_data[["Upg_or_Real", "Cancel_Rsn_ID"]].fillna(value = " ", inplace = True )

rsrvs_data["Upg_or_Real"].fillna(value = " ", inplace = True )
rsrvs_data["Cancel_Rsn_ID"].fillna(value = " ", inplace = True )

In [66]:

rsrvs_data[rsrvs_data["Room_Type"].isna( )]["Room_Type"]

2751      NaN
2864      NaN
29481     NaN
30904     NaN
30997     NaN
         ... 
241290    NaN
241730    NaN
241731    NaN
241814    NaN
241855    NaN
Name: Room_Type, Length: 5357, dtype: object

In [71]:
rsrvs_data[rsrvs_data["Board_ID"].isna( )]["Board_ID"].shape

(5260,)

In [74]:
print(f"Reservation Count : {rsrvs_data.shape } , Hueres Data Count : { hueres_data.shape }" )
guest_data = pd.merge(rsrvs_data,hueres_data, on=["Hotel_ID", "Reservation_ID"])

Reservation Count : (1063191, 13) , Hueres Data Count : (1579448, 4)


## Start of Analysis

### Analysis 1 : Total number of stays (visits)

In [81]:
# Count the number of reservations per guest
guest_data.groupby("Guest_ID").count( )["Reservation_ID"].sort_values(ascending = False )

,Hotel_ID,Reservation_ID,Main_Cust_ID,Checkin_Date,Checkout_Date,PAX,Room_Type,Upg_or_Real,Reservation_Status,Board_ID,Cancel_Date,Cancel_Rsn_ID,Reg_Date,Pers_pos
Guest_ID,,,,,,,,,,,,,,
0,221599,221599,221599,221599,221599,221599,221427,221599,221599,221599,221599,221599,221599,221599
1,166,166,166,166,166,166,166,166,166,166,166,166,166,166
2,51,51,51,51,51,51,51,51,51,51,51,51,51,51
3,12,12,12,12,12,12,11,12,12,11,12,12,12,12
7,4,4,4,4,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90450995,3,3,3,3,3,3,3,3,3,3,3,3,3,3
90451050,1,1,1,1,1,1,0,1,1,0,1,1,1,1
90451158,2,2,2,2,2,2,2,2,2,2,2,2,2,2


### Analysis 2: Number of unique hotels visited at the guest level.

In [96]:
# Count the number of hotels  per guest
hueres_data.groupby("Guest_ID").count( )["Hotel_ID"].sort_values(ascending = False )

Guest_ID
0           221599
3373          5048
1065593       2533
1216726        887
2716857        565
             ...  
3270710          1
3270709          1
3270708          1
3270707          1
90800342         1
Name: Hotel_ID, Length: 920102, dtype: int64

In [95]:
# kna1_data.info( )

## Some of the Guest IDs Not visible in KNA1 table!


kna1_data[kna1_data["Cust_ID"] == 2716857 ]
# kna1_data[kna1_data["Cust_ID"] == 1216726]

,Cust_ID,Country_ID,Name,City,Region,Gender,Company
24428,2716857,IN,Vijaya Bank,Jaipur,20,Mr.,
12233,2716857,IN,Vijaya Bank,Jaipur,20,Mr.,
41889,2716857,IN,Vijaya Bank,Jaipur,20,Mr.,


### Analysis 4 : Travel alone or with an Additional Guest

In [113]:
retail_guest_data = guest_data[(guest_data["Main_Cust_ID"] >= 0) & (guest_data["Main_Cust_ID"] < 90400000)][["Guest_ID", "PAX"]]
retail_guest_data

,Guest_ID,PAX
0,0,2
1,0,2
2,0,2
3,0,2
4,0,2
...,...,...
1579443,3339696,2
1579444,3339700,2
1579445,3339628,1
1579446,3339680,1


### Analysis 5: Determine Guest (Family / Groups)

In [112]:
retail_guest_data["Family"] = retail_guest_data["PAX"] > 1
retail_guest_data

,Guest_ID,PAX,Family
0,0,2,True
1,0,2,True
2,0,2,True
3,0,2,True
4,0,2,True
...,...,...,...
1579443,3339696,2,True
1579444,3339700,2,True
1579445,3339628,1,False
1579446,3339680,1,False


### Analysis 6 : Total number of Room nights across stays (for 2 years)

In [125]:
# guest_data["Stay_Length"] = pd.to_datetime(guest_data["Checkout_Date"]).apply(lambda x : x.date()) - pd.to_datetime(guest_data["Checkin_Date"]).apply(lambda x : x.date().days)

guest_data["Checkout_Date"] = pd.to_datetime(guest_data["Checkout_Date"], dayfirst=True).apply(lambda x : x.date())
guest_data["Checkin_Date"] = pd.to_datetime(guest_data["Checkin_Date"], dayfirst=True).apply(lambda x : x.date())
guest_data["Stay_Length"] = (guest_data["Checkout_Date"] - guest_data["Checkin_Date"]).apply(lambda x : x.days )
guest_data

,Hotel_ID,Reservation_ID,Main_Cust_ID,Checkin_Date,Checkout_Date,PAX,Room_Type,Upg_or_Real,Reservation_Status,Board_ID,Cancel_Date,Cancel_Rsn_ID,Reg_Date,Pers_pos,Guest_ID,Stay_Length
0,AGR1,3681294,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,1,0,1
1,AGR1,3681294,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,2,0,1
2,AGR1,3681296,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,1,0,1
3,AGR1,3681296,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,2,0,1
4,AGR1,3681298,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579443,VTZ1,3717484,90005463,2018-06-30,2018-01-07,2,TWN,STD,3,ONLYROOM,00.00.0000,,30.06.2018,1,3339696,-174
1579444,VTZ1,3717484,90005463,2018-06-30,2018-01-07,2,TWN,STD,3,ONLYROOM,00.00.0000,,30.06.2018,2,3339700,-174
1579445,VTZ1,3717543,90005463,2018-06-30,2018-01-07,1,SSR,,3,ONLYROOM,00.00.0000,,30.06.2018,1,3339628,-174
1579446,VTZ1,3717596,90005463,2018-06-30,2018-02-07,1,SSR,,3,ONLYROOM,00.00.0000,,30.06.2018,1,3339680,-143


In [127]:
# Sum the reservation length per guest to total stay length in past 2 years. 
guest_data.groupby("Guest_ID").sum( )["Stay_Length"]

Guest_ID
0           478719
1             -263
2             -154
3               79
7                4
             ...  
90450995         0
90451050         0
90451158         0
90452317         1
90800342        31
Name: Stay_Length, Length: 920102, dtype: int64

### Analysis 7 : Total Number of Unique Guests

In [129]:
hueres_data["Guest_ID"].nunique()

920102

### Analysis 8 : Total Number of Reservations

In [131]:
rsrvs_data["Reservation_ID"].nunique()

1063191

### Analysis 9 : Average Length of stay 

In [132]:
guest_data.groupby("Guest_ID").sum( )["Stay_Length"] / guest_data.groupby("Guest_ID").count( )["Reservation_ID"]

Guest_ID
0            2.160294
1           -1.584337
2           -3.019608
3            6.583333
7            1.000000
              ...    
90450995     0.000000
90451050     0.000000
90451158     0.000000
90452317     0.142857
90800342    31.000000
Length: 920102, dtype: float64

### Analysis 11 : Number & Share of Long Stays (>=5)


In [134]:
guest_data[guest_data["Stay_Length"] > 5]

,Hotel_ID,Reservation_ID,Main_Cust_ID,Checkin_Date,Checkout_Date,PAX,Room_Type,Upg_or_Real,Reservation_Status,Board_ID,Cancel_Date,Cancel_Rsn_ID,Reg_Date,Pers_pos,Guest_ID,Stay_Length
52,AGR1,3681356,90003186,2019-09-03,2019-10-03,2,DBL,,6,ONLYROOM,02.03.2019,1,05.06.2018,1,0,30
53,AGR1,3681356,90003186,2019-09-03,2019-10-03,2,DBL,,6,ONLYROOM,02.03.2019,1,05.06.2018,2,0,30
54,AGR1,3681357,90003186,2019-09-03,2019-10-03,2,DBL,,6,ONLYROOM,02.03.2019,1,05.06.2018,1,0,30
55,AGR1,3681357,90003186,2019-09-03,2019-10-03,2,DBL,,6,ONLYROOM,02.03.2019,1,05.06.2018,2,0,30
56,AGR1,3681358,90003186,2019-09-03,2019-10-03,2,DBL,,6,ONLYROOM,02.03.2019,1,05.06.2018,1,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579429,VTZ1,3716626,90400034,2018-06-30,2018-07-31,2,STD,,3,ONLYROOM,00.00.0000,,04.06.2018,1,3235321,31
1579430,VTZ1,3716626,90400034,2018-06-30,2018-07-31,2,STD,,3,ONLYROOM,00.00.0000,,04.06.2018,2,3266986,31
1579433,VTZ1,3716764,90400034,2018-06-30,2018-07-31,1,STD,,3,ONLYROOM,00.00.0000,,30.06.2018,1,2682790,31
1579434,VTZ1,3716887,90400034,2018-06-30,2018-08-08,2,STD,SSR,3,ONLYROOM,00.00.0000,,30.06.2018,1,929415,39


### Analysis 12/13 : Number & Share of Day Use

In [135]:
guest_data[guest_data["Stay_Length"] > 0]

,Hotel_ID,Reservation_ID,Main_Cust_ID,Checkin_Date,Checkout_Date,PAX,Room_Type,Upg_or_Real,Reservation_Status,Board_ID,Cancel_Date,Cancel_Rsn_ID,Reg_Date,Pers_pos,Guest_ID,Stay_Length
0,AGR1,3681294,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,1,0,1
1,AGR1,3681294,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,2,0,1
2,AGR1,3681296,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,1,0,1
3,AGR1,3681296,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,2,0,1
4,AGR1,3681298,90003186,2019-02-16,2019-02-17,2,DBL,,6,ONLYROOM,23.01.2019,1,05.06.2018,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579429,VTZ1,3716626,90400034,2018-06-30,2018-07-31,2,STD,,3,ONLYROOM,00.00.0000,,04.06.2018,1,3235321,31
1579430,VTZ1,3716626,90400034,2018-06-30,2018-07-31,2,STD,,3,ONLYROOM,00.00.0000,,04.06.2018,2,3266986,31
1579433,VTZ1,3716764,90400034,2018-06-30,2018-07-31,1,STD,,3,ONLYROOM,00.00.0000,,30.06.2018,1,2682790,31
1579434,VTZ1,3716887,90400034,2018-06-30,2018-08-08,2,STD,SSR,3,ONLYROOM,00.00.0000,,30.06.2018,1,929415,39
